In [73]:
import os
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
os.chdir("C:/Users/BabyPink/Documents/Estudio_UBER/")
df = pd.read_csv("dataframes/preparados/df_uber.csv", sep=",", parse_dates= ["date"],
                        dtype={"sourceid": np.int16, "dstid":np.int16, "day_name":np.int8 , "cat_time_range":np.int8,
                                "mean_travel_time":np.float32, "standard_deviation_travel_time": np.float32, 
                                "dist_kms":np.float32 , "vel_kms_hr": np.float32, "cancelado":np.int8, "resultado":np.int8},
                        memory_map=True,
                        usecols=['date', 'day_name', 'cat_time_range','mean_travel_time'],
                        index_col=False)

In [3]:
# Degrees of Freedom

In [4]:
N = len(df.mean_travel_time)
df_a = len(df.cat_time_range.unique()) - 1
df_b = len(df.day_name.unique()) - 1
df_axb = df_a*df_b
df_w = N - (len(df.cat_time_range.unique())*len(df.day_name.unique()))

In [5]:
# Sum of Squares

In [6]:
grand_mean = df['mean_travel_time'].mean()
grand_mean

11.479108

In [7]:
# Sum of Squares A (cat_time_range)

In [7]:
ssq_a = sum([df.groupby('cat_time_range')["mean_travel_time"].count()[j]*(df.groupby('cat_time_range')["mean_travel_time"].mean()[j]-grand_mean)**2 for j in range(len(df.cat_time_range.unique()))])
ssq_a


7562203.88638781

In [9]:
# Sum of Squares B (day_name)

In [8]:
ssq_b = sum([df.groupby('day_name')["mean_travel_time"].count()[j]*(df.groupby('day_name')["mean_travel_time"].mean()[j]-grand_mean)**2 for j in range(len(df.day_name.unique()))])
ssq_b

13886537.707660919

In [11]:
# Sum of Squares Total

In [69]:
ssq_t = ((df.mean_travel_time - grand_mean)**2).sum()
ssq_t

1171823000.0

In [13]:
# Sum of Squares Within (error/residual)

In [29]:
mad=df[df.cat_time_range == 0]
man=df[df.cat_time_range == 1]
med=df[df.cat_time_range == 2]
Tar=df[df.cat_time_range == 3]
noc=df[df.cat_time_range == 4]

In [34]:
mad_day_media = [mad[mad.day_name == d].mean_travel_time.mean() for d in range(len(df.day_name.unique()))]
man_day_media = [mad[mad.day_name == d].mean_travel_time.mean() for d in range(len(df.day_name.unique()))]
med_day_media = [mad[mad.day_name == d].mean_travel_time.mean() for d in range(len(df.day_name.unique()))]
Tar_day_media = [mad[mad.day_name == d].mean_travel_time.mean() for d in range(len(df.day_name.unique()))]
noc_day_media = [mad[mad.day_name == d].mean_travel_time.mean() for d in range(len(df.day_name.unique()))]


In [65]:
ssq_w_0_dd=sum([sum((mad[mad.day_name == j].mean_travel_time - mad_day_media[j])**2) for j in range (len(df.day_name.unique()))])
ssq_w_1_dd=sum([sum((man[man.day_name == j].mean_travel_time - mad_day_media[j])**2) for j in range (len(df.day_name.unique()))])
ssq_w_2_dd=sum([sum((med[med.day_name == j].mean_travel_time - mad_day_media[j])**2) for j in range (len(df.day_name.unique()))])
ssq_w_3_dd=sum([sum((Tar[Tar.day_name == j].mean_travel_time - mad_day_media[j])**2) for j in range (len(df.day_name.unique()))])
ssq_w_4_dd=sum([sum((noc[noc.day_name == j].mean_travel_time - mad_day_media[j])**2) for j in range (len(df.day_name.unique()))])


In [66]:
ssq_w = ssq_w_0_dd+ssq_w_1_dd+ssq_w_2_dd+ssq_w_3_dd+ssq_w_4_dd
ssq_w

1251139997.8723295

In [ ]:
# Sum of Squares interaction

In [70]:
ssq_axb = ssq_t-ssq_a-ssq_b-ssq_w
ssq_axb

-100765763.46637821

In [ ]:
# Mean Squares

In [71]:
# Mean Square A
ms_a = ssq_a/df_a

# Mean Square B
ms_b = ssq_b/df_b

# Mean Square AxB
ms_axb = ssq_axb/df_axb

# Mean Square Within/Error/Residual
ms_w = ssq_w/df_w




In [ ]:
# F-ratio

In [72]:
f_a = ms_a/ms_w
f_b = ms_b/ms_w
f_axb = ms_axb/ms_w

In [ ]:
# Obtaining p-values

In [74]:
p_a = stats.f.sf(f_a, df_a, df_w)
p_b = stats.f.sf(f_b, df_b, df_w)
p_axb = stats.f.sf(f_axb, df_axb, df_w)

In [ ]:
# ANOVA table

In [76]:
results = {'sum_sq':[ssq_a, ssq_b, ssq_axb, ssq_w],
           'df':[df_a, df_b, df_axb, df_w],
           'F':[f_a, f_b, f_axb, 'NaN'],
            'PR(&gt;F)':[p_a, p_b, p_axb, 'NaN']}
columns=['sum_sq', 'df', 'F', 'PR(&gt;F)']
aov_table1 = pd.DataFrame(results, columns=columns,
                          index=['cat_time_range', 'day_name', 
                          'cat_time_range:day_name', 'Residual'])

aov_table1

,sum_sq,df,F,PR(&gt;F)
cat_time_range,7.562204e+06,4,44918.907287,0.0
day_name,1.388654e+07,6,54989.974633,0.0
cat_time_range:day_name,-1.007658e+08,24,-99756.809321,1.0
Residual,1.251140e+09,29726700,NaN,NaN


In [27]:
medias_=df.groupby(["cat_time_range", "day_name"])["mean_travel_time"].mean()

In [39]:
medias_

cat_time_range  day_name
0               0            9.513391
                1            8.610188
                2           10.808045
                3            9.584452
                4           10.284332
                5           11.781633
                6           12.172630
1               0           12.098429
                1           11.280171
                2           11.317348
                3           11.708821
                4           12.334876
                5            8.737559
                6            7.904319
2               0           11.564168
                1           11.475990
                2           11.685236
                3           11.915915
                4           13.071247
                5           11.776781
                6           10.234344
3               0           11.175935
                1           11.293541
                2           11.770928
                3           12.009940
                4        

In [21]:
[df[df.cat_time_range==0].mean_travel_time]

[5           16.709999
 6           14.780000
 12          19.150000
 26          15.920000
 34          16.090000
               ...    
 29726727    18.370001
 29726728    22.760000
 29726731    15.460000
 29726732    26.940001
 29726733    23.959999
 Name: mean_travel_time, Length: 5972301, dtype: float32]

In [ ]:
df.groupby(["cat_time_range", 'day_name'])["mean_travel_time"].mean()[0][0]

In [15]:
df.columns

Index(['Unnamed: 0', 'len', 'supp', 'dose'], dtype='object')